# Prova de Conceito: Transcrição, NER e Geração de Ata
Este notebook realiza uma prova de conceito simples para transcrição de áudio, identificação de entidades nomeadas (NER) e geração de ata. Utilizaremos um arquivo de áudio de uma entrevista do Prof. Hendrik como exemplo.

In [2]:
# Instalar as dependências necessárias
!pip install openai-whisper transformers bert-extractive-summarizer protobuf sentencepiece tqdm

In [1]:
# Configuração do ambiente
# Importação das bibliotecas necessárias
import os
from pathlib import Path
import whisper
from transformers import pipeline
from summarizer import Summarizer

# Verificar se os modelos necessários estão instalados
print("Configuração inicial concluída.")

d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuração inicial concluída.


## Carregamento do Áudio
Carregamos o arquivo de áudio da entrevista para realizar a transcrição.

In [2]:

audio_path = Path("../data/raw/audio/entrevista_ufs_fm.mp3")

# Verificar se o arquivo existe
if not audio_path.exists():
    raise FileNotFoundError(f"Arquivo de áudio não encontrado: {audio_path}")

print(f"Arquivo carregado: {audio_path}")

Arquivo carregado: ..\data\raw\audio\entrevista_ufs_fm.mp3


## Transcrição do Áudio
Utilizamos o modelo Whisper para transcrever o áudio.

In [5]:
# Carregar o modelo Whisper
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")
model = whisper.load_model("small", device=device)

# Transcrever o áudio
result = model.transcribe(str(audio_path))
transcription = result["text"]

print("Transcrição concluída:")
print(transcription[:500])  # Exibir os primeiros 500 caracteres

Usando dispositivo: cpu


d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcrição concluída:
 A Universidade Federal de Sejip aprovou recentemente a criação do curso de graduação em inteligência artificial. Quem vai falar sobre o assunto a partir de agora com a gente aqui na UFIS FM é o professor Hendrik Macedo do Departamento de Computação da UFIS. Na satisfação ouvir o professor aqui na UFIS FM como surgiu a proposta de criação desse curso de bacharelado em IA. Boa tarde. Boa tarde, Josaphá. Eu e o que agradeço a oportunidade de esclarecer esse assunto. Veja bem, nós temos associados 


## Nomeação de Entidades (NER)
Identificamos entidades nomeadas na transcrição utilizando um modelo pré-treinado.

In [6]:
# Carregar o pipeline de NER
ner_pipeline = pipeline("ner", model="lfcc/bert-portuguese-ner", grouped_entities=True)

d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [7]:
# Função para dividir o texto em blocos menores
def split_text(text, max_length=350):
    words = text.split()
    for i in range(0, len(words), max_length):
        yield " ".join(words[i:i + max_length])

# Dividir a transcrição em blocos menores
text_blocks = list(split_text(transcription, max_length=350))

# Aplicar NER em cada bloco
entities = []
for block in text_blocks:
    entities.extend(ner_pipeline(block))

print("Entidades identificadas:")
for entity in entities:
    print(entity)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entidades identificadas:
{'entity_group': 'Organizacao', 'score': 0.8274329, 'word': 'Universidade Federal de Sejip', 'start': 2, 'end': 31}
{'entity_group': 'Organizacao', 'score': 0.5980847, 'word': 'UF', 'start': 182, 'end': 184}
{'entity_group': 'Organizacao', 'score': 0.4919821, 'word': '##IS', 'start': 184, 'end': 186}
{'entity_group': 'Pessoa', 'score': 0.9830049, 'word': 'Hend', 'start': 204, 'end': 208}
{'entity_group': 'Pessoa', 'score': 0.9813471, 'word': '##ri', 'start': 208, 'end': 210}
{'entity_group': 'Pessoa', 'score': 0.96631706, 'word': '##k Macedo', 'start': 210, 'end': 218}
{'entity_group': 'Organizacao', 'score': 0.50499254, 'word': 'UF', 'start': 252, 'end': 254}
{'entity_group': 'Organizacao', 'score': 0.5281395, 'word': '##IS', 'start': 254, 'end': 256}
{'entity_group': 'Organizacao', 'score': 0.6223447, 'word': 'UF', 'start': 298, 'end': 300}
{'entity_group': 'Organizacao', 'score': 0.54624575, 'word': '##IS', 'start': 300, 'end': 302}
{'entity_group': 'Pessoa'

## Geração de Ata
Com base na transcrição e nas entidades, geramos um resumo estruturado.

In [8]:
# Carregar o modelo de sumarização
# summarizer = pipeline("summarization", model="pierreguillou/t5-base-pt-sum-cnndm-242M")
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "recogna-nlp/ptt5-base-summ"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Gerar resumo
summary = summarizer(transcription, max_length=500, min_length=50, do_sample=False)

print("Resumo gerado:")
print(summary)

Resumo gerado:
[{'summary_text': "'Não vão aprender a usar chat APT', diz professor da UFIS. A universidade quer que o curso de inteligência artificial seja um dos mais concorridos do Brasil. E a gente tem uma preocupação com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estr